In [25]:
import pandas as pd
import numpy as np
import requests
import time
import re
import json
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
%matplotlib inline
from collections import defaultdict

In [2]:
test = pd.read_csv("helper/test.csv")
train = pd.read_csv("helper/train.csv")
artists = pd.read_csv("helper/artists.csv")
profiles = pd.read_csv("helper/profiles.csv")

In [3]:
train.head()

user  \
0  eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03   
1  44ce793a6cd9d20f13f4a576a818ef983314bb5d   
2  da9cf3f557161d54b76f24db64be9cc76db008e3   
3  8fa49ab25d425edcf05d44bfc1d5aea895287d81   
4  b85fcaef67d2669cd99b334b5e8c8705263db2cf   

                                 artist  plays  
0  5a8e07d5-d932-4484-a7f7-e700793a9c94    554  
1  a3a92047-be1c-4f3e-8960-c4f8570984df     81  
2  eeb1195b-f213-4ce1-b28c-8565211f8e43    708  
3  a1419808-65d3-4d40-998c-1a0bac65eabc    265  
4  a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432    220

In [4]:
artists.head()

artist                     name
0  03098741-08b3-4dd7-b3f6-1b0bfa2c879c                    Liars
1  69c4cc43-8163-41c5-ac81-30946d27bb69          CunninLynguists
2  7a2e6b55-f149-4e74-be6a-30a1b1a387bb      The Desert Sessions
3  7002bf88-1269-4965-a772-4ba1e7a91eaa              Glenn Gould
4  dbf7c761-e332-467b-b4d9-aafe06bbcf8f  G. Love & Special Sauce

In [5]:
profiles.head()

user sex  age        country
0  fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d   f   25         Sweden
1  5909125332c108365a26ccf0ee62636eee08215c   m   29        Iceland
2  d1867cbda35e0d48e9a8390d9f5e079c9d99ea96   m   30  United States
3  63268cce0d68127729890c1691f62d5be5abd87c   m   21        Germany
4  02871cd952d607ba69b64e2e107773012c708113   m   24    Netherlands

In [6]:
GLOBAL_MEAN = np.mean(train.plays)
GLOBAL_MEDIAN = np.median(train.plays)
artist_list = list(set(artists.name))

In [7]:
'''grouped_by_user = train.groupby('user')
user_means = grouped_by_user['plays'].mean()
user_medians = grouped_by_user['plays'].median()
user_stds = grouped_by_user['plays'].std()
user_ids = user_means.index

user_stats = {}
for i in range(len(grouped_by_user)) :
    user_stats[user_ids[i]] = {
        'mean' : user_means[i],
        'std' : user_stds[i],
        'median' : user_medians[i]
    }

with open('helper/user_stats.txt', 'w') as outfile:
    json.dump(user_stats, outfile)'''
    
with open('helper/user_stats.txt') as data_file:    
    user_stats = json.load(data_file)
    
print "Successfully loaded " + str(len(user_stats.keys())) + " user profiles"

Successfully loaded 233286 user profiles


In [8]:
'''grouped_by_artist = train.groupby('artist')
artist_means = grouped_by_artist['plays'].mean()
artist_medians = grouped_by_artist['plays'].median()
artist_stds = grouped_by_artist['plays'].std()
artist_ids = artist_means.index

artist_stats = {}
for i in range(len(grouped_by_artist)) :
    artist_stats[artist_ids[i]] = {
        'mean' : artist_means[i],
        'std' : artist_stds[i],
        'median' : artist_medians[i]
    }

artist_count = 0
wiki_count = 0'''

print "Unique artists: " + str(len(artist_list))

Unique artists: 1978


In [9]:
'''def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def get_wiki_url(artist, search_engine) :
    if pd.isnull(artist) == False and is_ascii(artist) :
        fixArtist = artist.replace(" ","+").replace("&","%26")
        
        if search_engine == "Yahoo" :
            var = requests.get('https://search.yahoo.com/search;_ylt=A0oG7l7PeB5P3G0AKASl87UF?p=site:wikipedia.org+'
                               +fixArtist+'+music+band+artist')
        elif search_engine == "Bing" :
            var = requests.get('http://www.bing.com/search?q=site%3Awikipedia.org+'
                               +fixArtist+'+music+band+artist')
        elif search_engine == "AOL" :
            var = requests.get('http://search.aol.com/aol/search?enabled_terms=&s_it=comsearch&q=site%3Awikipedia.org+'
                               +fixArtist+"+music+band+artist")
        else :
            sys.exit("Out of search engines!")
                               
        soup = BeautifulSoup(var.text, "html.parser")
        relevant = [link for link in [el.get("href") for el in soup.find_all('a')] 
                    if type(link) == unicode and re.search('^https://en.wikipedia.org/', link)]
        if len(relevant) > 0 :
            return relevant[0]
        else :
            return "Unknown"

    else :
        return "Unknown"

if artist_count < len(artist_list) :
    for j in range(artist_count,len(artist_list)) :
        artist_name = str(artist_list[j])
        if artist_name != "nan" :
            artist_id = artists[artists.name == artist_name]['artist'].values[0]
            try : 
                artist_stats[artist_id]['url'] = get_wiki_url(artist_name, "Bing")
            except :
                break
            artist_stats[artist_id]['name'] = artist_name
        artist_count += 1
else :
    print "Already finished"
    
missing_artists = []
for key in artist_stats.keys() :
    if len(artist_stats[key]) != 4 :
        missing_artists.append(key)

for missing_id in missing_artists :
    artist_name = artists[artists.artist == missing_id]['name'].values[0]
    try : 
        artist_stats[missing_id]['url'] = get_wiki_url(artist_name, "Bing")
    except :
        break
        
if wiki_count < len(artist_stats.keys()) :
    for key in artist_stats.keys()[counter:len(artist_stats.keys())] :
        url = artist_stats[key]['url']
        if url != "Unknown" :
            var = requests.get(url)
            soup = BeautifulSoup(var.text, "html.parser")

            genre_exists = soup.find(title='Music genre')
            if genre_exists :
                artist_stats[key]['genre'] = [entry.get("href") 
                                              for entry in soup.find(title="Music genre").parent.parent.find_all('a')[1:] 
                                              if entry.get("href")[1:5] == 'wiki']
            else :
                artist_stats[key]['genre'] = "Unknown"

            ya_exists = soup.find(text=re.compile("Years"))
            try :
                artist_stats[key]['ya'] = ya_exists.parent.parent.find('td').get_text()
            except AttributeError :
                artist_stats[key]['ya'] = "Unknown"
        else :
            artist_stats[key]['genre'] = 'Unknown'
            artist_stats[key]['ya'] = 'Unknown'
        print wiki_count
        counter += 1
else :
    print "Already done!"

with open('helper/artist_stats.txt', 'w') as outfile:
    json.dump(artist_stats, outfile)'''

with open('helper/artist_stats.txt') as data_file:    
    artist_stats = json.load(data_file)
    
print "Successfully loaded " + str(len(artist_stats.keys())) + " artist profiles"

Successfully loaded 2000 artist profiles


In [10]:
# Why are there more profiles than names? 
print "Number of unique IDs: " + str(len(np.unique(artists.artist)))
print 

print "List of duplicates:"
artistDict = defaultdict(int)
for name in artists.name :
    artistDict[name] += 1
    if artistDict[name] > 1 and pd.isnull(name) == False:
        print "- " + name
        

Number of unique IDs: 2000

List of duplicates:
- Justice
- Ben Harper
- Camera Obscura
- Nouvelle Vague
- Enigma
- Nirvana
- Rhapsody of Fire
- Moby
- Feist
- [Disney]
- [unknown]
- M.I.A.
- Air


In [11]:
# Looks like there are some duplicates ... 
artists[artists.name == 'Justice']

artist     name
72  444f2191-8841-440e-bbc1-6fd564893946  Justice
80  f9c1cc73-36ba-4f8d-ba12-7d2490608886  Justice

In [12]:
# The reslts are similar but not exactly the same
print artist_stats['444f2191-8841-440e-bbc1-6fd564893946']
print 
print artist_stats['f9c1cc73-36ba-4f8d-ba12-7d2490608886']

{u'std': 214.25874253893798, u'name': u'Justice', u'url': u'https://en.wikipedia.org/wiki/Justice_%28French_band%29', u'median': 114.0, u'ya': u'2003\u2013present', u'genre': [u'/wiki/Electro_house', u'/wiki/Electronica', u'/wiki/Nu-disco', u'/wiki/Electroclash', u'/wiki/Electronic_rock', u'/wiki/Alternative_dance'], u'mean': 175.85247275775356}

{u'std': 257.9073693778499, u'name': u'Justice', u'url': u'https://en.wikipedia.org/wiki/Justice_%28French_band%29', u'median': 109.0, u'ya': u'2003\u2013present', u'genre': [u'/wiki/Electro_house', u'/wiki/Electronica', u'/wiki/Nu-disco', u'/wiki/Electroclash', u'/wiki/Electronic_rock', u'/wiki/Alternative_dance'], u'mean': 178.7460778617083}


In [13]:
artist_df = pd.DataFrame.from_dict(artist_stats, orient='index')
artist_df.reset_index(level=0, inplace=True)
artist_df.columns = artist_df.columns.str.replace('index','artist_id')
print artist_df.shape

(2000, 8)


In [14]:
possible_genres = defaultdict(int)

for artist_genres in artist_df.genre :
    if artist_genres == u"Unknown" :
        possible_genres[u"Unknown"] += 1
    elif type(artist_genres) == list :
        for genre in artist_genres :
            possible_genres[genre] += 1
            
print "Unique genres: " + str(len(possible_genres.keys()))

Unique genres: 1709


In [15]:
import operator
sorted_genres = sorted(possible_genres.items(), key=operator.itemgetter(1), reverse=True)
filtered_genres = [zip[0] for zip in sorted_genres[0:251]]
for genre in filtered_genres :
    artist_df[genre] = [genre in song_genres if type(song_genres) != bool else False for song_genres in artist_df.genre]
artist_df = artist_df.drop([u'/wiki/Wikipedia:Citation_needed'], axis=1)
print artist_df.shape

(2000, 258)


In [16]:
start_year = []
active = []

for years in artist_df.ya :
    if years != "Unknown" :
        if len(years) > 4 and ("19" in years[0:4] or "20" in years[0:4]):
            result = int(years[0:4])
            if 1900 < result < 2016 :
                start_year.append(result)
            else :
                start_year.append("Unknown")
        else :
            start_year.append("Unknown")
            
        if "present" in years :
            active.append("Active")
        else :
            active.append("Nonactive")
    else :
        start_year.append("Unknown")
        active.append("Unknown")

artist_df['start_year'] = start_year
artist_df['active'] = active

print artist_df.shape

(2000, 260)


In [17]:
artist_df = artist_df.drop(['genre','ya'],axis=1)
artist_df.to_pickle('helper/artist_df.pkl')
artist_df.head()

artist_id         std                      name  \
0  000d90ec-d64c-48a1-b775-e726fd240e9f  249.016613  Get Cape. Wear Cape. Fly   
1  000fc734-b7e1-4a01-92d1-f544261b43f5  661.689032             Cocteau Twins   
2  0019749d-ee29-4a5f-ab17-6bfa11deb969  353.443050                   DJ Food   
3  0039c7ae-e1a7-4a7d-9b49-0cbc716821a6  631.262555       Death Cab for Cutie   
4  004e5eed-e267-46ea-b504-54526f1f377d  835.701307             The Gathering   

                                                 url  median        mean  \
0  https://en.wikipedia.org/wiki/Get_Cape._Wear_C...   121.5  196.296078   
1        https://en.wikipedia.org/wiki/Cocteau_Twins   137.0  288.887880   
2  https://en.wikipedia.org/wiki/List_of_Brazilia...    69.5  145.452174   
3  https://en.wikipedia.org/wiki/Death_Cab_for_Cutie   176.0  341.798469   
4        https://en.wikipedia.org/wiki/Magic!_(band)   164.0  370.565190   

  /wiki/Alternative_rock Unknown /wiki/Indie_rock /wiki/Pop_music    ...      \
0                  False   False             True           False    ...       
1                  False   False            False           False    ...       
2                  False    True            False           False    ...       
3                   True   False             True           False    ...       
4                  False   False            False            True    ...       

  /wiki/Sophisti-pop /wiki/New_prog /wiki/Acid_techno /wiki/Electro_music  \
0              False          False             False               False   
1              False          False             False               False   
2              False          False             False               False   
3              False          False             False               False   
4              False          False             False               False   

  /wiki/P-Funk /wiki/Synthesizer /wiki/Hip_house /wiki/Sampling_(music)  \
0        False             False           False                  False   
1        False             False           False                  False   
2        False             False           False                  False   
3        False             False           False                  False   
4        False             False           False                  False   

  start_year     active  
0       2004     Active  
1       1979  Nonactive  
2    Unknown    Unknown  
3       1997     Active  
4       2012     Active  

[5 rows x 258 columns]